In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
import urllib
import urllib.request
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common import exceptions
import warnings
import re
warnings.filterwarnings('ignore')
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

In [2]:
city = 'south goa'

In [3]:
url = 'https://www.google.com/travel/hotels?hrf=YAA&ictx=3&destination='+ city + '&ap=MAFaoQEKBQiIJxAAIgNJTlIqFgoHCOUPEAIYGBIHCOUPEAIYGRgBKACwAQBYAWAAaAGaAS4SBVN1cmF0GiUweDNiZTA0ZTU5NDExZDE1NjM6MHhmZTQ1NTgyOTA5MzhiMDQyogESCgkvbS8wMWgxaG4SBVN1cmF0qgELCgIIEhIDCJsBGAGqAQoKAgguEgIIPRgBqgELCgMI4QISAghjGAGSAQIgAWgA&rp=SAI&ts=CAESABpQCjISLjIlMHgzYmUwNGU1OTQxMWQxNTYzOjB4ZmU0NTU4MjkwOTM4YjA0MjoFU3VyYXQaABIaEhQKBwjlDxACGBgSBwjlDxACGBkYATICEAAqDwoLKAhKAiABOgNJTlIaAA&ved=0CAAQ5JsGahgKEwjYh5_g-P_uAhUAAAAAHQAAAAAQhQI'

In [4]:
options = Options()
options.add_argument('--headless')
options.add_argument('--lang=en')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome(chrome_options=options)
driver.get(url)

In [5]:
listOflinks=[]
condition=True
while condition:
    listlinks=driver.find_elements_by_class_name('f1dFQe')
    for elm in listlinks:
        pp2=elm.find_element_by_tag_name('a')
        listOflinks.append(pp2.get_attribute('href'))
        time.sleep(5)
    try:
        
        kk= driver.find_elements_by_class_name('RveJvd.snByac')[-2]
        print(kk.text)
        if kk.text == 'Next':
            kk.click()
        else:
            condition=False
        time.sleep(5)       
    except:
        condition=False

Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Next
Previous


In [6]:
len(listOflinks)

330

In [10]:
scraped_data = []
for l in listOflinks:
        response = requests.get(l)
        data = bs(response.text, 'html.parser')
        hotel_details = data.find_all('div', attrs={'class': 'iInyCf QqZUDd'})
                           
        for hotel in hotel_details:

            address_details ={}

            hotel_name = hotel.find('h1', attrs={'class':'fZscne'})
                                                       
            #condition = True
            #while condition:
            
            prices_button = driver.find_elements_by_xpath('//*[@id="overview"]/c-wiz/div/div[2]/div/div/div[4]/section/div[2]/c-wiz/div[2]/div/button')
            time.sleep(3)
            for b in prices_button:
                b.click()
            #else:
                #condition = False
                
            booking_details = data.find_all('div', attrs={'class':'KoLVjf bM3xBe'})
            
            for booking in booking_details:             

                    hotel_booking = booking.find_all('span', attrs={'class':'FjC1We ogfYpf zUyrwb'})
                    hotel_price = booking.find_all('span', attrs={'class':'MW1oTb'})

            hotel_type = hotel.find_all('span', attrs={'class':'CFH2De'})
            for t in hotel_type:
                address_details['hotel_type']=t.text
            hotel_address = hotel.find('div', attrs={'class':'K4nuhf'})
            hotel_phone = hotel.find_all('span', {'dir':'ltr'})
            for p in hotel_phone:
                address_details['hotel_phone']=p.text
            hotel_website = hotel.find_all('a', {'class': 'FKF6mc TpQm9d'})[0]['href']           
            hotel_rating = hotel.find('div', attrs={'class':'iDqPh BgYkof'})
            hotel_reviews_count = hotel.find('a', attrs={'class':'eS7K5e'})
            hotel_description = hotel.find_all('div', attrs={'class':'yIr6Rd'})
            for d in hotel_description:
                address_details['hotel_description']=d.text
                      
            address_details['hotel_booking']= hotel_booking
            address_details['hotel_price']=hotel_price
            address_details['hotel_name']=hotel_name.text
            
            address_details['hotel_address']=hotel_address.text
            address_details['hotel_website']=hotel_website
            address_details['hotel_rating']=hotel_rating
            address_details['hotel_reviews_count']=hotel_reviews_count
            #address_details['hotel_emails']=hotel_emails
                      
            scraped_data.append(address_details)
               
        dataFrame2 = pd.DataFrame.from_dict(scraped_data)

In [11]:
dataFrame2

,hotel_type,hotel_phone,hotel_booking,hotel_price,hotel_name,hotel_address,hotel_website,hotel_rating,hotel_reviews_count,hotel_description
0,098221 53527,098221 53527,"[[Booking.com], [Agoda], [Priceline], [MobyTrip]]","[[₹2,240], [₹2,240], [₹2,240], [₹2,225]]",Sao Domingo's Holiday Home,"House no 354(A-1 Mobor, Cavelossim, Goa 403731...",http://saodomingosgoa.com/,[4.4],[36 reviews],NaN
1,094208 20470,094208 20470,"[[Booking.com], [Agoda], [Priceline], [MobyTrip]]","[[₹1,210], [₹1,452], [₹1,613], [₹2,016], [₹2,8...",Jardim a Mar,"Agonda Beach Rd, Agonda, Goa 403702•094208 20470",http://www.jardim-a-mar.com/,[4.3],[168 reviews],Simple resort offering no-frills rooms & low-k...
2,097644 42775,097644 42775,"[[Booking.com], [Yatra.com], [Tripadvisor.in],...","[[₹1,000], [₹1,680], [₹2,240], [₹2,800], [₹2,8...",Palolem Beach Resort,"Palolem - Patnem Beach Road Patnem, Canacona, ...",http://www.cubagoa.com/,[4.2],"[3,096 reviews]",Relaxed rooms and cottages in a hotel facing t...
3,091584 91489,091584 91489,"[[Booking.com], [trivago.in], [Agoda], [Pricel...","[[₹4,011], [₹5,157], [₹4,870], [₹6,016], [₹5,9...",Blue Lagoon Cola Resort,"Cola Beach Road, Canacona, Goa 403702•091584 9...",http://www.bluelagoonresortgoa.com/,[4.3],[530 reviews],Secluded cottages in a tranquil beachfront get...
4,0832 672 6100,0832 672 6100,"[[Booking.com], [trivago.in], [Priceline], [Mo...","[[₹1,656], [₹2,199], [₹2,575], [₹2,558], [₹2,1...",Karma Royal Palms,"Colva - Benaulim Rd, Benaulim, Goa 403576•0832...",https://karmagroup.com/find-destination/karma-...,[4.3],"[4,938 reviews]",Casual studios & apartments in a resort featur...
...,...,...,...,...,...,...,...,...,...,...
325,"Palolem Down St, Palolem, Canacona, Goa 403702",NaN,[],[],Yoga Culture,"Palolem Down St, Palolem, Canacona, Goa 403702",http://www.yogatreeculture.com/,[5.0],[1 review],NaN
326,0124 620 1615,0124 620 1615,[],[],OYO 18661 Peaceful Stay,"1, Fatrade Beach Rd, Fatrade, Varca, Goa 40372...",https://www.oyorooms.com/hotels-in-Goa/?utm_so...,[3.9],[13 reviews],NaN
327,"NEAR MADGAON RAILWAY STATION, ON MARGAO FIRE B...",NaN,[],[],SPOT ON 29574 Hotel Gold Star Inn,"NEAR MADGAON RAILWAY STATION, ON MARGAO FIRE B...",https://www.oyorooms.com/hotels-in-Goa/?utm_so...,[2.7],[16 reviews],NaN
328,0124 620 1517,0124 620 1517,[[Agoda]],"[[₹1,539]]",SPOT ON 77526 Goa Star,"Aquem, Margao, Goa 403601•0124 620 1517",https://www.oyorooms.com/h/108685/?utm_source=...,[2.0],[1 review],NaN


In [32]:
scraped_data2 =[]
for h in dataFrame2['hotel_website']:
    address_details ={}
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
    try:
        response = requests.request('GET', h, headers=headers, timeout=6.0)
    except (ConnectTimeout, HTTPError, ReadTimeout, Timeout, ConnectionError):
        continue
        
    data = bs(response.text, 'html.parser')
    the_page = data.decode('utf-8')
    email_pattern = re.compile(r'''([-a-zA-Z0-9._+-]+@[-a-zA-Z0-9._-]+.[-a-zA-Z]+)''', re.VERBOSE)
    phone_pattern = re.compile(r'''(?:\s+|)((0|(?:(\+|)91))(?:\s|-)*(?:(?:\d(?:\s|-)*\d{9})|(?:\d{2}(?:\s|-)*\d{8})|(?:\d{3}(?:\s|-)*\d{7}))|\d{10})(?:\s+|)''', re.VERBOSE)
    emails = re.findall(email_pattern, the_page)
    phone =  re.findall(phone_pattern, the_page)
    address_details['hotel_emails']= emails
    address_details['hotel_phone']= phone
    
    
    scraped_data2.append(address_details)
    dataFrame3 = pd.DataFrame.from_dict(scraped_data2)

In [63]:
dataFrame3['hotel_website']=dataFrame2['hotel_website']

In [64]:
dataFrame3

,hotel_emails,hotel_phone,hotel_website
0,[],"[(5641025641, , ), (7179487179, , ), (81196581...",http://saodomingosgoa.com/
1,"[info@cubagoa.com, info@cubagoa.com, info@cuba...","[(09764442775, 0, ), (+91 9764442775, +91, +),...",http://www.jardim-a-mar.com/
2,"[bluelagoonresortgoa@gmail.com, bluelagoonreso...","[(+91 9158491489, +91, +), (1544434255, , ), (...",http://www.cubagoa.com/
3,"[royalpalms@karmagroup.com, royalpalms@karmagr...","[(3600000000, , ), (3600000000, , ), (99999999...",http://www.bluelagoonresortgoa.com/
4,"[reservations@gracehospitality.in, reservation...","[(+91 832 2770461, +91, +), (+91 832 2770661, ...",https://karmagroup.com/find-destination/karma-...
...,...,...,...
314,"[robert@broofa.com\nCopyright, robert@broofa.c...","[(00999999977, 0, ), (00100000004, 0, ), (1000...",http://seagullhotelpalolem.com/
315,"[rubanyoga@gmail.com, rubanyoga@gmail.com]","[(9370322700, , ), (5719936783, , ), (15432338...",https://maps.google.com/maps?gl=IN&hl=en-US&um...
316,"[bookings@oyorooms.com, bookings@oyorooms.com,...","[(1160274348, , ), (2728440072, , ), (15764753...",https://www.goa-tourism.com/old_goa_resi
317,[john@email.com],"[(1160274348, , ), (2728440072, , ), (15764753...",https://www.oyorooms.com/hotels-in-Goa/?utm_so...


In [65]:
dataFrame2.to_csv('hotels_project7.csv', index=False)

In [66]:
dataFrame3.to_csv('hotel_emails_phone2.csv', index=False)